In [ ]:
import numpy as np
import shapely
import geopandas as gpd
import contextily as cx

from shapely import Point, LineString, Polygon

import matplotlib.pyplot as plt

# 1. Generating target geometry

In [ ]:
# Parameters
AREA = 500_000_000_000 # m^2
NUM_LANES = 4
D_HUBS = 50_000 # m
D_LANES = 5 # m

CENTER = shapely.Point(0, 0)
RADIUS = np.sqrt(AREA / np.pi)
BOUNDS = CENTER.buffer(RADIUS)

In [ ]:
seq = [D_HUBS, *[D_LANES] * (NUM_LANES - 1)]
seq = [0] + seq * int(np.floor(2 * RADIUS / np.sum(seq)))
seq = np.cumsum(seq)
seq = seq - np.max(seq) / 2

In [ ]:
def consecutive_pairs(s):
    it = iter(s)
    next(it)
    return list(zip(s, it))

In [ ]:
points = [Point(x, y) for x in seq for y in seq]

In [ ]:
lines = []

for a, b in consecutive_pairs([-2 * RADIUS, *seq, 2 * RADIUS]):
    lines.extend(LineString(((x, a), (x, b))) for x in seq)
    lines.extend(LineString(((a, y), (b, y))) for y in seq)

In [ ]:
polygons = []

for a, b in consecutive_pairs([-2 * RADIUS, *seq, 2 * RADIUS]):
    for c, d in consecutive_pairs([-2 * RADIUS, *seq, 2 * RADIUS]):
        polygons.append(Polygon(((a, c), (a, d), (b, d), (b, c))))

In [ ]:
def create_gdf(geometry, rotation=45):
    """
    Create a GeoDataFrame from a list of shapely geometries.
    """
    return gpd.GeoDataFrame(geometry=geometry, crs='epsg:3857') \
        .intersection(BOUNDS) \
        .to_crs('epsg:4326')

In [ ]:
gdf_points   = create_gdf(points)
gdf_lines    = create_gdf(lines)
gdf_polygons = create_gdf(polygons)

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(10, 10))
gdf_points.plot(ax=axs[0])
gdf_lines.plot(ax=axs[1])
gdf_polygons.plot(ax=axs[2])

for ax in axs:
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
import os
os.makedirs('shape/points', exist_ok=True)
os.makedirs('shape/lines', exist_ok=True)
os.makedirs('shape/polygons', exist_ok=True)
gdf_points.to_file('shape/points/points.shp')
gdf_points.to_file('shape/lines/lines.shp')
gdf_points.to_file('shape/polygons/polygons.shp')

# 2. Generating points